Courtesy of Andrej Karpathy

In [ ]:
from graphviz import Digraph

In [ ]:
from retrograd.engine import Value
from retrograd.neural import *

In [ ]:
def trace(root):
    nodes, edges = set(), set()
    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._prev:
                edges.add((child, v))
                build(child)
    build(root)
    return nodes, edges

def draw_dot(root, format='svg', rankdir='LR'):
    """
    format: png | svg | ...
    rankdir: TB (top to bottom graph) | LR (left to right)
    """
    assert rankdir in ['LR', 'TB']
    nodes, edges = trace(root)
    dot = Digraph(format=format, graph_attr={'rankdir': rankdir}) #, node_attr={'rankdir': 'TB'})
    
    for n in nodes:
        dot.node(name=str(id(n)), label = "{ %s | data %.4f | grad %.4f }" % (n.label, n.data, n.grad), shape='record')
        if n._op:
            dot.node(name=str(id(n)) + n._op, label=n._op)
            dot.edge(str(id(n)) + n._op, str(id(n)))
    
    for n1, n2 in edges:
        dot.edge(str(id(n1)), str(id(n2)) + n2._op)
    
    return dot

In [ ]:
# a very simple example
# x = Value(4.0,label='x')
# y = (x*2 - 4*x + x**2)
# y.label='y'
# y.backward()
# draw_dot(y)

In [147]:
# MLP
x = [1. , 2. , 3.]
nn = MLP(3,[4,4,1])
y = nn(x)
# y.backward()
# draw_dot(y)

In [148]:
example_dataset = [
  [2.0, 3.0, -1.0],
  [3.0, -1.0, 0.5],
  [0.5, 1.0, 1.0],
  [1.0,1.0,-1.0],
]
y_real = [1.0,-1.0,-1.0,1.0]

for k in range(50):
  #forward pass
  y_pred = [nn(x) for x in example_dataset]
  loss = sum([(y_p - y_r)**2 for y_p,y_r in zip(y_pred,y_real)])

  #zero grad
  for p in nn.get_parameters():
    p.grad = 0.0

  #backward pass
  loss.backward()
  
  #update
  for p in nn.get_parameters():
    p.data -= 0.1 * p.grad
  print(f'Step {k} | Loss: {loss.data}')